In [ ]:
import numpy as np
import pandas as pd
#from sklearn.cluster import KMeans
import scipy 
import sklearn
from collections import Counter
from sklearn.metrics import multilabel_confusion_matrix
from scipy import spatial
#Import data
#Split data
#Function Definitions start
#sse_calculation

#kMeans Algorithm
##Initialize centroids
##Looping through the data...
##Calculate distance of each point from the centroid using distance functions
##Assign that point to that centroid whose distance is minimum
##Calucate new centroids for each cluster
##Iterate until the centroids doesnt change

#Calculate SSE for the entire data points
#Compare between different distance measures

#Accuracy calculation for the clusters
#Compare accuracies between various Kmeans algorithms




In [ ]:
data = pd.read_csv('data.csv')
labels = pd.read_csv('label.csv',names=['label'],header=None)

In [ ]:
data.head()

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,...,0.658,0.659,0.660,0.661,0.662,0.663,0.664,0.665,0.666,0.667
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
data.count()

0        9999
0.1      9999
0.2      9999
0.3      9999
0.4      9999
         ... 
0.663    9999
0.664    9999
0.665    9999
0.666    9999
0.667    9999
Length: 784, dtype: int64

In [ ]:
#Split data
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split( data, test_size=0.08, random_state=42)
train_labels, test_labels = train_test_split( labels, test_size=0.08, random_state=42)

In [ ]:
class KMeans:
    
    def calculate_SSE(self, centroid_value_dict, centroid_dict,data):
        sse_data = 0
        for i in centroid_dict:
            sse_cluster = 0
            # np.sum()
            for j in centroid_dict[i]:
                dp = list(data.iloc[int(j)])
                for a,b in zip(centroid_value_dict[i],dp):
                    sse_cluster += (a-b)**2
            sse_data+=sse_cluster
        return sse_data    
    
    def Initialize_Centroids(self,data,K):
        m = data.shape[0]
        centroid_value_dict={}
        for i in range(K):
            r = np.random.randint(0, m-1)
            centroid_value_dict[i] = data.iloc[r]
        return centroid_value_dict
    
    def jaccard_similarity(self,centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

    def train_Kmeans(self,data,K,max_iter=20,mode=1,tol=10):
        #Mode = 1 => Euclidean np.linalg.norm(x-list(data.iloc[i,:]))
        #Mode = 2 => Jaccard
        #Mode = 3 => Cosine
        centroid_value_dict = self.Initialize_Centroids(data,K)
        new_centroid_value_dict = {}
        count = 0
        centroid_dict = {}
        convergence = False
        while((count<max_iter) and not convergence):
            
            for i in list(centroid_value_dict.keys()):
                centroid_dict[i]=[]
            for i in range(data.shape[0]):
                x = data.iloc[i]
                if mode==1 :
                    distance_measure = [np.linalg.norm(x-centroid_value_dict[j])  for j in centroid_value_dict]
                    idx = np.argmin(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==2 :
                    distance_measure = [self.jaccard_similarity(list(x),centroid_value_dict[j]) for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                elif mode==3 :
                    distance_measure = [1-scipy.spatial.distance.cosine(x,list(centroid_value_dict[j]))  for j in centroid_value_dict]
                    idx = np.argmax(distance_measure)
                    centroid_dict[idx].append(i)
                
                prev_centroids=dict(centroid_value_dict)
                
            
            for i in centroid_dict:
                if len(centroid_dict[i]):
                    dps_centroid = centroid_dict[i]
                    centroid_value_dict[i] = np.average(data.iloc[dps_centroid],axis=0)
            
            
            current_tol=-1
            for i in centroid_value_dict:
                prev_centroid_point = prev_centroids[i]
                new_centroid_point = centroid_value_dict[i]
                change = np.sum(np.absolute(new_centroid_point-prev_centroid_point))
                current_tol = max(change, current_tol)
                
            print("Tolerance for the Iteration ",count,": ",current_tol)
            
            count+=1
            if (current_tol<10):
                convergence = True
                break
           # print("KMeans Iteration",count)
        return centroid_value_dict,centroid_dict
    

In [ ]:
def predict_cluster_labels(C, S, labels):
    '''
    Input : C -> Centroids
            S -> Set of Indicies corresponding to Centroid C
            data -> Data used to form clusters
    Output : Returns an array of size K having labels based on majority voting in the cluster
    '''
    cluster_labels = np.zeros(10,dtype=int)
    for c in C:
        labels_of_points = []
        for point in S[c]:
            labels_of_points.extend(labels.iloc[point])
        counter = Counter(labels_of_points)
        try:
            cluster_labels[c] = max(counter, key=counter.get)
        except:
            cluster_labels[c] = np.random.randint(0,9)
    return cluster_labels

In [ ]:
def jaccard_similarity(centroid, dp):
        intersection = len(list(set(centroid).intersection(dp)))
        union = (len(set(centroid)) + len(set(dp))) - intersection
        return float(intersection) / union

In [ ]:
def accuracy(centroids, centroid_Labels, test_data, true_labels, mode=1):
    y_true = list(true_labels['label']);
    y_pred = []
    for index in range(test_data.shape[0]):
        featureset = test_data.iloc[index]
        if mode==1:
            distances = [np.linalg.norm(featureset - centroids[centroid]) for centroid in centroids]
            classification = distances.index(min(distances))
            y_pred.append(centroid_Labels[classification])
        elif mode==2:
            similarity = [jaccard_similarity(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification]) 
        elif mode==3:
            similarity = [1 - spatial.distance.cosine(featureset, centroids[centroid]) for centroid in centroids]
            classification = similarity.index(max(similarity))
            y_pred.append(centroid_Labels[classification])
    denominator = test_data.shape[0]
    correctly_classified = 0
    for i in range(0,len(y_pred)):
        if y_true[i] == y_pred[i]:
            correctly_classified += 1
    accuracy = correctly_classified/denominator
    return accuracy

In [ ]:
model1 = KMeans()
centroids1,clusters1 = model1.train_Kmeans(data,10, max_iter=100,mode=1)

Tolerance for the Iteration  0 :  25399.763837638377
Tolerance for the Iteration  1 :  5475.672324324324
Tolerance for the Iteration  2 :  3693.2691114006902
Tolerance for the Iteration  3 :  1965.8205126729877
Tolerance for the Iteration  4 :  1233.4706540115094
Tolerance for the Iteration  5 :  1051.99305376852
Tolerance for the Iteration  6 :  1253.2210376234252
Tolerance for the Iteration  7 :  1777.2707764422803
Tolerance for the Iteration  8 :  1857.8346894599547
Tolerance for the Iteration  9 :  1282.4874629435933


In [ ]:
Euclidean_SSE = model1.calculate_SSE(centroids1,clusters1,data)

In [ ]:
print("Euclidean SSE:",Euclidean_SSE)

Euclidean SSE: 25623612263.443344


In [ ]:
cluster_labels1 = predict_cluster_labels(centroids1,clusters1,labels)
cluster_labels1

array([5, 7, 1, 0, 9, 7, 2, 3, 2, 3])

In [ ]:
Accuracy_Euclidean = accuracy(centroids1, cluster_labels1,test_data,test_labels)
Accuracy_Euclidean

0.0875

In [ ]:
model2 = KMeans()
centroids2,clusters2 = model2.train_Kmeans(data,10, max_iter=100,mode=2)
Jaccard_SSE = model2.calculate_SSE(centroids2,clusters2,data)

Tolerance for the Iteration  0 :  31041.110842541435
Tolerance for the Iteration  1 :  5701.428019406059
Tolerance for the Iteration  2 :  5903.836702390525
Tolerance for the Iteration  3 :  3200.5725967333574
Tolerance for the Iteration  4 :  1763.0961547678314
Tolerance for the Iteration  5 :  1651.631984750199
Tolerance for the Iteration  6 :  2149.0778014577013
Tolerance for the Iteration  7 :  1528.8817131042863
Tolerance for the Iteration  8 :  1274.728399160671
Tolerance for the Iteration  9 :  895.3300161559778
Tolerance for the Iteration  10 :  1249.6638385868973
Tolerance for the Iteration  11 :  733.8049140542282
Tolerance for the Iteration  12 :  298.1460533833932
Tolerance for the Iteration  13 :  0.0


In [ ]:
print("Jacard SSE:",Jaccard_SSE)

Jacard SSE: 34361687572.938736


In [ ]:
cluster_labels2 = predict_cluster_labels(centroids2,clusters2,labels)
cluster_labels2

array([1, 7, 8, 6, 3, 4, 2, 5, 2, 2])

In [ ]:
Accuracy_Jaccard = accuracy(centroids2, cluster_labels2,test_data,test_labels)
Accuracy_Jaccard

0.1075

In [ ]:
model3 = KMeans()
centroids3,clusters3 = model3.train_Kmeans(data,10, max_iter = 100,mode=3)

Tolerance for the Iteration  0 :  28857.200934579436
Tolerance for the Iteration  1 :  6231.249798135506
Tolerance for the Iteration  2 :  3137.6074955127924
Tolerance for the Iteration  3 :  2924.342337606713
Tolerance for the Iteration  4 :  2587.3035629200167
Tolerance for the Iteration  5 :  2187.340578534064
Tolerance for the Iteration  6 :  1742.6148843930632
Tolerance for the Iteration  7 :  1341.8570176643248
Tolerance for the Iteration  8 :  1158.6871033753755
Tolerance for the Iteration  9 :  932.8574325041448
Tolerance for the Iteration  10 :  813.5194476789702
Tolerance for the Iteration  11 :  761.6655889262166
Tolerance for the Iteration  12 :  697.9690828630726
Tolerance for the Iteration  13 :  659.5169542250799
Tolerance for the Iteration  14 :  711.2075496503007
Tolerance for the Iteration  15 :  686.3316840899992
Tolerance for the Iteration  16 :  473.8157045299995
Tolerance for the Iteration  17 :  322.77061030113487
Tolerance for the Iteration  18 :  319.1711558819

In [ ]:
Cosine_SSE = model3.calculate_SSE(centroids3,clusters3,data)

In [ ]:
print("Euclidean SSE:",Euclidean_SSE)
print("Jacard SSE:",Jaccard_SSE)
print("Cosine SSE :",Cosine_SSE)

Euclidean SSE: 25623612263.443344
Jacard SSE: 34361687572.938736
Cosine SSE : 25645122363.170094


In [ ]:
cluster_labels3 = predict_cluster_labels(centroids3,clusters3,labels)
cluster_labels3

array([5, 3, 2, 4, 3, 9, 6, 8, 1, 0])

In [ ]:
#Accuracy_Cosine = accuracy(centroids3, cluster_labels3,test_data,test_labels)
Accuracy_Euclidean
Accuracy_Jaccard
Accuracy_Cosine

0.085

In [ ]:
print("Euclidean accuracy:",Accuracy_Euclidean)
print("Jacard accuracy:",Accuracy_Jaccard)
print("Cosine accuracy :",Accuracy_Cosine)

Euclidean accuracy: 0.0875
Jacard accuracy: 0.1075
Cosine accuracy : 0.085
